<a href="https://colab.research.google.com/github/syedaftab-dev/Machine-learning/blob/main/adaboost2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Random forest project - Regression

---

# Later we implemented Adaboost Regressor



In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [22]:
df=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/cardekho_imputated.csv",index_col=[0])

In [23]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Data Cleaning

In [24]:
# check missing aval
df.isnull().sum()

,0
car_name,0
brand,0
model,0
vehicle_age,0
km_driven,0
seller_type,0
fuel_type,0
transmission_type,0
mileage,0
engine,0


In [25]:
# 3 columns giving same data so delete other
df.drop('car_name',axis=1,inplace=True)
df.drop('brand',axis=1,inplace=True)


In [26]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [27]:
# geting all diffeernt type of feature
num_features = [feature for feature in df.columns if df[feature].dtype !='O']
print(num_features)
cat_features = [feature for feature in df.columns if df[feature].dtype =='O']
print(cat_features)
discreate_feature = [feature for feature in df.columns if len(df[feature].unique())<=25]
print(discreate_feature)
continuous_feature = [feature for feature in df.columns if feature not in discreate_feature]
print(continuous_feature)

['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats', 'selling_price']
['model', 'seller_type', 'fuel_type', 'transmission_type']
['vehicle_age', 'seller_type', 'fuel_type', 'transmission_type', 'seats']
['model', 'km_driven', 'mileage', 'engine', 'max_power', 'selling_price']


In [28]:
## independent and dependent feature
X=df.drop(['selling_price'],axis=1); # contains all features other than selling_price
y=df['selling_price']


# Feature Encoding and Scaling

# Label encoding (just like onehotencoding but when there lot of different categories we use this one)
In a dataset with a Fruit column containing "Apple," "Banana," and "Orange," label encoding can map :

Apple -> 0

Banana -> 1

Orange -> 2

In [29]:
len(X['model'].unique())


120

In [30]:
X['model'].value_counts()

,count
model,
i20,906
Swift Dzire,890
Swift,781
Alto,778
City,757
...,...
Altroz,1
C,1
Ghost,1


In [31]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])
# label-encoding will give like aulto-0,i20-2,swift-3 all the categories are represented by numerical


In [32]:
# create columnTranformer with 3 types of transformer

num_features = X.select_dtypes(exclude='object').columns
onehot_columns = ['seller_type','fuel_type','transmission_type']

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer =  OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,onehot_columns),
        ("StandardScaler",numeric_transformer,num_features)

    ],remainder='passthrough' # insure not to remove the other remaining columns
)

In [33]:
X=preprocessor.fit_transform(X)


In [34]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Model Training and model selection

In [43]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [37]:
def evaluate_model(true,predicted):
  mae=mean_absolute_error(true,predicted)
  mse=mean_squared_error(true,predicted)
  rmse=np.sqrt(mean_squared_error(true,predicted))
  r2=r2_score(true,predicted)
  return mae,rmse,r2

In [44]:
# begining model training
models={
    'Random Forest':RandomForestRegressor(),
    'Linear Regression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'K-Neighbors Regressor':KNeighborsRegressor(),
    'Decision Tree':DecisionTreeRegressor(),
    'AdaBoost Regressor':AdaBoostRegressor()
}

for i in range(len(list(models))):
  model=list(models.values())[i]
  model.fit(X_train,y_train)

  # make predictions
  y_train_pred=model.predict(X_train)
  y_test_pred=model.predict(X_test)

  y_train_pred=model.predict(X_train)
  y_test_pred=model.predict(X_test)

  # evalaute train and test dataset
  model_train_mae,model_train_rmse,model_train_r2= evaluate_model(y_train,y_train_pred)
  model_test_mae,model_test_rmse,model_test_r2= evaluate_model(y_test,y_test_pred)

  print(list(models.keys())[i])

  print('Model performance for training set')
  print('- Root Mean Squared Error: {:.4f}'.format(model_train_rmse))
  print('- Mean Absolute Error: {:.4f}'.format(model_train_mae))
  print('- R2 Score: {:.4f}'.format(model_train_r2))

  print('----------------------------------')
  print('Model performance for test set')
  print('- Root Mean Squared Error: {:.4f}'.format(model_test_rmse))
  print('- Mean Absolute Error: {:.4f}'.format(model_test_mae))
  print('- R2 Score: {:.4f}'.format(model_test_r2))


Random Forest
Model performance for training set
- Root Mean Squared Error: 136822.4889
- Mean Absolute Error: 39933.3688
- R2 Score: 0.9769
----------------------------------
Model performance for test set
- Root Mean Squared Error: 226142.4660
- Mean Absolute Error: 101040.7427
- R2 Score: 0.9321
Linear Regression
Model performance for training set
- Root Mean Squared Error: 553855.6665
- Mean Absolute Error: 268101.6071
- R2 Score: 0.6218
----------------------------------
Model performance for test set
- Root Mean Squared Error: 502543.5930
- Mean Absolute Error: 279618.5794
- R2 Score: 0.6645
Lasso
Model performance for training set
- Root Mean Squared Error: 553855.6710
- Mean Absolute Error: 268099.2226
- R2 Score: 0.6218
----------------------------------
Model performance for test set
- Root Mean Squared Error: 502542.6696
- Mean Absolute Error: 279614.7461
- R2 Score: 0.6645
Ridge
Model performance for training set
- Root Mean Squared Error: 553856.3160
- Mean Absolute Error:

# HYperparameter tuning

In [45]:
knn_params = {"n_neighbors":[2,3,10,20,40,50]}
rf_params={
    "max_depth":[5,8,15,None,10],
    "max_features":[5,7,"auto",8],
    "min_samples_split":[2,8,15,20],
    "n_estimators":[100,200,500,1000],
}
ada_params={
    'n_estimators':[50,60,70,80],
    'loss':['linear', 'square', 'exponential'] # loss function
}

In [46]:
# models list for hyperparamter tuninih
randomcv_models = [
    ('KNN',KNeighborsRegressor(),knn_params),
    ('RF',RandomForestRegressor(),rf_params),
    ('Adaboost',AdaBoostRegressor(),ada_params)
    ]

In [47]:
from sklearn.model_selection import RandomizedSearchCV

model_param={}
for model_name,model,params in randomcv_models:
  random=RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
  random.fit(X_train,y_train)
  model_param[model_name]=random.best_params_

for model_name in model_param:
  print(f"Best params for {model_name} is {model_param[model_name]}")

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best params for KNN is {'n_neighbors': 10}
Best params for RF is {'n_estimators': 500, 'min_samples_split': 2, 'max_features': 5, 'max_depth': 15}
Best params for Adaboost is {'n_estimators': 60, 'loss': 'linear'}


# *We will get the best params from above cell take those paramter and train the model*